In [ ]:
from qat.lang.AQASM import Program, H, CNOT, RX, RY, RZ, X, Z
import numpy as np
from qat.qpus import PyLinalg
from numpy import linalg
from qat.qpus import get_default_qpu
from qat.core import Observable, Term

# pyqubo imports
from pyqubo import Binary, Spin
from pprint import pprint
from dimod import ExactSolver

import scipy
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Hands-on: Problema da Mochila no VQE

## Descrição do problema

Imagine que voce tem uma mochila com capacidade $W$ (peso total) e barras de ouro de diversos pesos, $w_i$. Você quer colocar o máximo de ouro dentro da mochila, sem que sua capacidade seja ultrapassada. Quais barras de ouro colocar de forma a maximizar a quantidade de ouro dentro da mochila? Esta é uma versão simplificada do problema. Este exemplo pode ser generalizado para várias mochilas e objetos com valores e pesos diferentes.

<img src="mochila.jpeg" width=750 />

## Formulação como otimização irrestrita

Como queremos maximizar a quantidade de ouro dentro da mochila, podemos criar uma função objetivo de minimização da seguinte forma

$$H(\vec{x}) = \left(W - \sum_{i=0}^{n}w_ix_i\right)^2$$
onde $x_i=1$ se a barra de ouro $i$ está dentro da mochila e $x_i=0$ caso contrário. 





## Hands-on

Vamos fazer um exemplo simples, com poucas variáveis. Imagine que 3 clientes te pediram barras de um determinado material, cujos comprimentos são: Cliente 1 - `5m`; Cliente 2 - `3m`; e Cliente 3 - `1m`. No entando, você tem apenas uma barra com `6m`. Como cortar a barra de `6m` de forma a minimizar a sobra. Use o VQE para solucionar o problema.

### Exercício 1

Crie a função objetivo do problema usando o `Pyqubo`. Lembre-se de usar a classe `Binary("xi")` como suas variáveis binárias.

In [ ]:
# -----------------------------
# criando a F.O. com pyqubo
# -----------------------------

# escreva seu codigo abaixo
num_variables = FIXME

# [...]

H = FIXME
# fim do seu codigo

# -----------------------------------
# Transformando de Binary para Ising
# -----------------------------------

model = H.compile()
model = model.to_ising()
linear, quadratic = model[0], model[1]

print("Termos lineares do modelo de Ising", linear)
print("Termos cruzados do modelo de Ising", quadratic)

### Exercício 2

Escreva o modelo de Ising em termos de operadores de Pauli-Z. Crie um dicionário para armazenar cada termo da seguinte forma:

Exemplo: `hamiltonian = {"ZIZ": 4, ...}` onde as chaves são strings com operadores de Pauli-Z e Identidade e os respectivos valores são seus coeficientes.

In [ ]:
# escreva o codigo abaixo
  
hamiltonian = FIXME
# fim do seu codigo

print(hamiltonian)

Agora, escreva o operador na forma matricial. Dica: crie as matrizes $Z$ e $I$ e use np.kron() para fazer o produto tensorial.

In [ ]:
# Escreva seu codigo abaixo

h_matrix = FIXME
# fim do seu codigo

### Ansatz

O ansatz é um circuito quântico parametrizado. Sua forma variacional pode ser dada por padrões heurísticos geralmente implementada através de operadores de rotação e portas CNOT para realizar emaranhamento entre os qubits. Uma forma variacional eficiente é aquela capaz de generalizar bem um estado quântico, aumentando o espaço de busca. Podemos escrever a atuação do circuito variacional, $U(\vec{\theta})$, sobre um sistema com $n$ qubits de estado inicial, $|0\rangle^{\otimes n}$, como

$$U(\vec{\theta})|0\rangle^{\otimes n} = |\psi(\vec{\theta})\rangle$$

Agora, realizaremos o mesmo processo do exemplo anterios, mas agora não estaremos interessados em ter um número como solução, e sim um vetor, o $\vec{z}$. Embora minimizaremos o valor esperado de um operador, estaremos interessados no autoestado com maior probabilidade, associado ao menor valor esperado.

### Exercício 3

Aplique as operações no ansatz e defina, no lugar do `FIXME`, o numero de parametros variacionais que voce usou.

In [ ]:
def ansatz(var_params):

    qprog = Program()
    qbits = qprog.qalloc(num_variables)

    # escreva seu codigo abaixo
    

    # fim do seu codigo
  
    circ = qprog.to_circ()
    return circ

num_params = FIXME

circuit = ansatz([i for i in range(num_params)])
%qatdisplay circuit --svg

### Exercício 4

Na função de calculo do valor esperado, defina os termos do Observável. Cada termo deve ser escrito como `Term(coeficiente, operador, [0,1,2])` e devem estar dentro da lista `pauli_terms`.

In [ ]:
def expected_value(params):

    # chamando o ansatz
    circuit = ansatz(var_params = params)
    
    # criando o observavel
    # a classe Oservable é responsavel por realizar as medidas em diferentes bases
    # cada termo do observavel deve ser definido como Term(coeficiente, operador, [0,1,2]) 
    pauli_terms = [ FIXME ]
    obs = Observable(3, pauli_terms = pauli_terms)
                                    
    # calcula o valor esperado do observavel para um conjunto de parametros
    job = circuit.to_job(observable=obs, nbshots=1000)
    result = get_default_qpu().submit(job)
    print(" CUSTO: \t ", result.value)
    conv.append(result.value)

    return result.value

def run_vqe_comb():

    res = scipy.optimize.minimize(expected_value, x0=np.ones(num_params), 
                                method = 'COBYLA', callback=None,
                                options={'maxiter': 500, 'ftol': 1e-06, 'iprint': 20, 'disp': True, 
                                'eps': 1.4901161193847656e-08, 'finite_diff_rel_step': None})

    print("\n" , 100*"-",  "\n Aproximação da Energia Fundamental: \t ", res['fun'], "\n", 100*"-")
    print("\n Parametros ótimos do circuito varicional: \t ", res['x'])

    return res['x'], conv

conv = []
best_params, conv = run_vqe_comb()

### Exercicio 5

Pegue os melhores parametros encontrados na minimização do valor esperado e aplique-os no ansatz. Este circuito será o circuito "ótimo" e medidas neste circuito nos dará informações sobre as melhores soluções.

In [ ]:
# corrija os FIXMEs aqui
best_circuit = FIXME
job = FIXME.to_job()
# fim do exercicio

result = get_default_qpu().submit(job)

states, probs = [], []
for sample in result:
    print(f"Estado:  {sample.state} \t Probabilidade:  {sample.probability}")
    states.append(sample.state)
    probs.append(sample.probability)

print(f" \n \n Autoestado com maior ocorrência:  {str(states[probs.index(max(probs))])} \t Probabilidade:  {max(probs)}")

O autoestado com maior ocorrência indica a solução ótima.

In [ ]:
plt.plot(conv)
plt.xlabel("Iteracao")
plt.ylabel("Valor Esperado <H>")
plt.grid()